# Process Optimization and Prediction

## Purpose

This notebook teaches you how to implement process optimization and prediction for additive manufacturing processes. You'll learn to build predictive quality models, perform early defect detection, optimize process parameters (single and multi-objective), validate optimization results, and track model performance using a unified interactive interface with real-time progress tracking and detailed logging.

## Learning Objectives

By the end of this notebook, you will:
- ✅ Build predictive quality models (Random Forest, Gradient Boosting, MLP)
- ✅ Perform early defect detection before build completion
- ✅ Forecast quality metrics using time-series models (ARIMA, Exponential Smoothing, Moving Average, Prophet)
- ✅ Optimize process parameters (single-objective and multi-objective)
- ✅ Handle constraints in optimization (penalty, barrier, augmented Lagrangian)
- ✅ Validate optimization results (cross-validation, experimental, simulation)
- ✅ Track model performance over time with drift detection
- ✅ Register and version models in a model registry
- ✅ Execute complete end-to-end prediction and optimization workflows
- ✅ Monitor prediction and optimization progress with real-time status and logs

## Estimated Duration

90-120 minutes

---

## Overview

Process Optimization and Prediction enables data-driven optimization of manufacturing processes with predictive models. The AM-QADF framework provides comprehensive capabilities:

- 🔮 **Predictive Quality Models**: Random Forest, Gradient Boosting, MLP for quality prediction
- ⚠️ **Early Defect Detection**: Predict defects early in build process before completion
- 📈 **Time-Series Forecasting**: ARIMA, Exponential Smoothing, Moving Average, Prophet for quality and parameter forecasting
- 🎯 **Single-Objective Optimization**: Differential Evolution, L-BFGS-B for parameter optimization
- 📊 **Multi-Objective Optimization**: NSGA-II, Weighted Sum for Pareto front generation
- 🚧 **Constrained Optimization**: Penalty, Barrier, Augmented Lagrangian constraint handling
- ⚡ **Real-Time Optimization**: Streaming optimization with adaptive parameter updates
- ✅ **Optimization Validation**: Cross-validation, experimental, and simulation validation
- 📝 **Model Registry**: Version control and storage for trained models
- 📈 **Performance Tracking**: Monitor model performance, detect degradation and drift
- 🔍 **Model Monitoring**: Continuous monitoring with retraining triggers

The notebook features a unified interactive interface with:
- **Progress Tracking**: Visual progress bars showing completion percentage
- **Status Monitoring**: Real-time status updates with elapsed time
- **Detailed Logging**: Timestamped logs with success/warning/error indicators for all operations
- **Error Handling**: Comprehensive error messages and tracebacks in the logs

Use the interactive widgets below to build models, optimize parameters, and track performance - no coding required! Monitor your prediction and optimization progress in real-time using the status bar and logs section at the bottom.

## Interactive Process Optimization and Prediction Interface

Use the widgets below to build predictive models, perform early defect detection, forecast quality metrics, optimize process parameters, validate results, and track model performance. All prediction and optimization tasks are organized systematically in one unified interface!

In [1]:
# Setup: Import required libraries
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add parent directory and src directory to path for imports
notebook_dir = Path().resolve()
project_root = notebook_dir.parent
src_dir = project_root / 'src'

# Add project root to path (for src.infrastructure imports)
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Add src directory to path (for am_qadf imports)
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Core imports
import ipywidgets as widgets
from ipywidgets import (
    VBox, HBox, Accordion, Tab, Dropdown, RadioButtons, 
    Checkbox, Button, Output, Text, IntSlider, FloatSlider,
    Layout, Box, Label, FloatText, IntText, SelectMultiple,
    HTML as WidgetHTML, Textarea, FileUpload, Valid, Play, jslink
)
from IPython.display import display, Markdown, HTML, clear_output, Javascript
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import time
import json
import threading
from typing import Optional, Tuple, Dict, Any, List
import asyncio
from collections import deque
import tempfile
import shutil

# Set style for plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Load environment variables from development.env
import os
env_file = project_root / 'development.env'
if env_file.exists():
    with open(env_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '=' in line:
                key, value = line.split('=', 1)
                value = value.strip('"\'')
                os.environ[key] = value
    print("✅ Environment variables loaded from development.env")

# Try to import prediction classes
PREDICTION_AVAILABLE = False
try:
    from am_qadf.analytics.process_analysis.prediction.early_defect_predictor import (
        EarlyDefectPredictor, PredictionConfig, EarlyDefectPredictionResult
    )
    from am_qadf.analytics.process_analysis.prediction.time_series_predictor import (
        TimeSeriesPredictor, TimeSeriesPredictionResult
    )
    from am_qadf.analytics.process_analysis.prediction.prediction_validator import (
        PredictionValidator, OptimizationValidationResult
    )
    PREDICTION_AVAILABLE = True
    print("✅ Prediction classes available")
except ImportError as e:
    print(f"⚠️ Prediction classes not available: {e} - using demo mode")

# Try to import model tracking classes
MODEL_TRACKING_AVAILABLE = False
try:
    from am_qadf.analytics.process_analysis.model_tracking.model_registry import (
        ModelRegistry, ModelVersion
    )
    from am_qadf.analytics.process_analysis.model_tracking.performance_tracker import (
        ModelPerformanceTracker, ModelPerformanceMetrics
    )
    from am_qadf.analytics.process_analysis.model_tracking.model_monitor import (
        ModelMonitor, ModelMonitoringConfig
    )
    MODEL_TRACKING_AVAILABLE = True
    print("✅ Model tracking classes available")
except ImportError as e:
    print(f"⚠️ Model tracking classes not available: {e} - using demo mode")

# Try to import optimization classes
OPTIMIZATION_AVAILABLE = False
try:
    from am_qadf.analytics.process_analysis.optimization import (
        ProcessOptimizer, OptimizationConfig, OptimizationResult,
        ConstraintHandler, ParetoVisualizer
    )
    OPTIMIZATION_AVAILABLE = True
    print("✅ Optimization classes available")
except ImportError as e:
    print(f"⚠️ Optimization classes not available: {e} - using demo mode")

# Try to import quality analysis classes
QUALITY_AVAILABLE = False
try:
    from am_qadf.analytics.process_analysis.quality_analysis import (
        QualityPredictor, QualityAnalysisConfig, QualityAnalysisResult
    )
    QUALITY_AVAILABLE = True
    print("✅ Quality analysis classes available")
except ImportError as e:
    print(f"⚠️ Quality analysis classes not available: {e} - using demo mode")

# MongoDB connection setup (optional, for persistence)
INFRASTRUCTURE_AVAILABLE = False
mongo_client = None

try:
    from src.infrastructure.config import MongoDBConfig
    from src.infrastructure.database import MongoDBClient
    
    config = MongoDBConfig.from_env()
    if not config.username:
        config.username = os.getenv('MONGO_ROOT_USERNAME', 'admin')
    if not config.password:
        config.password = os.getenv('MONGO_ROOT_PASSWORD', 'password')
    
    mongo_client = MongoDBClient(config=config)
    try:
        if mongo_client.is_connected():
            INFRASTRUCTURE_AVAILABLE = True
            print(f"✅ Connected to MongoDB: {config.database}")
        else:
            mongo_client = None
            print("⚠️ MongoDB connection failed - using demo mode")
    except Exception as conn_error:
        mongo_client = None
        print(f"⚠️ MongoDB connection check failed: {conn_error} - using demo mode")
except ImportError as e:
    print(f"⚠️ MongoDB infrastructure not available: {e} - using demo mode")
    mongo_client = None
except Exception as e:
    print(f"⚠️ MongoDB not available: {e} - using demo mode")
    mongo_client = None

print("✅ Setup complete!")

✅ Environment variables loaded from development.env
✅ Prediction classes available
⚠️ Model tracking classes not available: cannot import name 'ModelMonitoringConfig' from 'am_qadf.analytics.process_analysis.model_tracking.model_monitor' (/mnt/c/Users/kanha/Independent_Research/AM-QADF/src/am_qadf/analytics/process_analysis/model_tracking/model_monitor.py) - using demo mode
✅ Optimization classes available
✅ Quality analysis classes available
✅ Connected to MongoDB: am_qadf_data
✅ Setup complete!


In [2]:
# Create Interactive Process Optimization and Prediction Interface

# Global state
prediction_results = {}
optimization_results = {}
model_registry = None
performance_trackers = {}
current_operation = None
operation_start_time = None
is_operation_active = False

# Initialize model registry if available
if MODEL_TRACKING_AVAILABLE:
    temp_registry_dir = tempfile.mkdtemp()
    model_registry = ModelRegistry(storage_path=temp_registry_dir)
    print("✅ Model registry initialized")
else:
    print("⚠️ Model registry not available - using demo mode")

# Initialize clients
quality_predictor = None
early_defect_predictor = None
time_series_predictor = None
prediction_validator = None
optimizer = None

if QUALITY_AVAILABLE:
    quality_config = QualityAnalysisConfig(random_seed=42)
    quality_predictor = QualityPredictor(quality_config)
    print("✅ Quality predictor initialized")

if PREDICTION_AVAILABLE:
    pred_config = PredictionConfig(random_seed=42)
    early_defect_predictor = EarlyDefectPredictor(pred_config)
    time_series_predictor = TimeSeriesPredictor(pred_config)
    prediction_validator = PredictionValidator(pred_config)
    print("✅ Prediction components initialized")

if OPTIMIZATION_AVAILABLE:
    opt_config = OptimizationConfig(random_seed=42)
    optimizer = ProcessOptimizer(opt_config)
    print("✅ Optimizer initialized")

# ============================================
# Helper Functions for Demo Data Generation
# ============================================

def generate_demo_process_data(n_samples=200, n_features=5):
    """Generate demo process data for prediction and optimization."""
    np.random.seed(42)
    
    # Generate process parameters
    data = {
        'laser_power': np.random.uniform(200, 300, n_samples),
        'scan_speed': np.random.uniform(800, 1200, n_samples),
        'layer_thickness': np.random.uniform(0.02, 0.04, n_samples),
        'hatch_spacing': np.random.uniform(0.08, 0.12, n_samples),
        'temperature': np.random.uniform(800, 1200, n_samples),
    }
    
    # Create quality as function of parameters
    quality = (
        0.7 +
        0.001 * data['laser_power'] +
        0.0002 * data['scan_speed'] -
        10 * data['layer_thickness'] -
        5 * data['hatch_spacing'] +
        np.random.randn(n_samples) * 0.05
    )
    quality = np.clip(quality, 0.0, 1.0)
    data['quality'] = quality
    
    # Create defect labels (1 if quality < 0.6)
    data['defect_label'] = (quality < 0.6).astype(int)
    
    # Add temporal structure
    data['timestamp'] = pd.date_range(start='2024-01-01', periods=n_samples, freq='H')
    data['build_id'] = [f'build_{i//50}' for i in range(n_samples)]
    
    return pd.DataFrame(data)

print("✅ Helper functions initialized")

⚠️ Model registry not available - using demo mode
✅ Quality predictor initialized
✅ Prediction components initialized
✅ Optimizer initialized
✅ Helper functions initialized


In [3]:
# ============================================
# Top Panel: Operation Type Selection and Actions
# ============================================

operation_type_label = WidgetHTML("<b>Operation Type:</b>")
operation_type = RadioButtons(
    options=[
        ('Predictive Quality Models', 'quality_prediction'),
        ('Early Defect Detection', 'early_defect'),
        ('Time-Series Forecasting', 'time_series'),
        ('Process Optimization (Single-objective)', 'optimization_single'),
        ('Process Optimization (Multi-objective)', 'optimization_multi'),
        ('Optimization Validation', 'validation'),
        ('Real-Time Optimization', 'realtime_optimization'),
        ('Model Tracking', 'model_tracking'),
        ('Complete Workflow', 'complete')
    ],
    value='quality_prediction',
    description='Type:',
    style={'description_width': 'initial'}
)

data_source_label = WidgetHTML("<b>Data Source:</b>")
data_source_mode = RadioButtons(
    options=[('Demo Data', 'demo'), ('MongoDB', 'mongodb'), ('CSV File', 'csv')],
    value='demo',
    description='Source:',
    style={'description_width': 'initial'}
)

execute_button = Button(
    description='Execute Operation',
    button_style='success',
    icon='play',
    layout=Layout(width='200px', height='40px')
)

stop_button = Button(
    description='Stop Operation',
    button_style='danger',
    icon='stop',
    layout=Layout(width='180px', height='40px'),
    disabled=True
)

export_button = Button(
    description='Export Results',
    button_style='',
    icon='download',
    layout=Layout(width='150px', height='40px')
)

top_panel = VBox([
    HBox([operation_type_label, operation_type], layout=Layout(margin='10px')),
    HBox([data_source_label, data_source_mode, execute_button, stop_button, export_button], 
         layout=Layout(margin='10px'))
], layout=Layout(border='2px solid #0277bd', padding='10px', margin='5px'))

print("✅ Top panel created")

✅ Top panel created


In [4]:
# ============================================
# Left Panel: Configuration Accordion
# ============================================

# Prediction Configuration
prediction_model_type = Dropdown(
    options=['random_forest', 'gradient_boosting', 'mlp'],
    value='random_forest',
    description='Model Type:',
    layout=Layout(width='100%')
)

prediction_features = SelectMultiple(
    options=['laser_power', 'scan_speed', 'layer_thickness', 'hatch_spacing', 'temperature'],
    value=['laser_power', 'scan_speed', 'layer_thickness', 'hatch_spacing'],
    description='Features:',
    layout=Layout(width='100%', height='100px')
)

train_test_split = FloatSlider(
    value=0.2,
    min=0.1,
    max=0.5,
    step=0.05,
    description='Train-Test Split:',
    layout=Layout(width='100%')
)

cv_folds = IntSlider(
    value=5,
    min=3,
    max=10,
    step=1,
    description='CV Folds:',
    layout=Layout(width='100%')
)

enable_early_prediction = Checkbox(
    value=True,
    description='Enable Early Prediction',
    layout=Layout(width='100%')
)

early_prediction_horizon = IntSlider(
    value=100,
    min=50,
    max=500,
    step=50,
    description='Early Horizon:',
    layout=Layout(width='100%')
)

enable_time_series = Checkbox(
    value=False,
    description='Enable Time-Series',
    layout=Layout(width='100%')
)

forecast_horizon = IntSlider(
    value=10,
    min=5,
    max=50,
    step=5,
    description='Forecast Horizon:',
    layout=Layout(width='100%')
)

prediction_config = VBox([
    WidgetHTML("<b>Prediction Configuration</b>"),
    prediction_model_type,
    prediction_features,
    train_test_split,
    cv_folds,
    enable_early_prediction,
    early_prediction_horizon,
    enable_time_series,
    forecast_horizon,
], layout=Layout(padding='10px'))

# Optimization Configuration
optimization_type = RadioButtons(
    options=[('Single-objective', 'single'), ('Multi-objective', 'multi')],
    value='single',
    description='Type:',
    style={'description_width': 'initial'}
)

optimization_method = Dropdown(
    options=['differential_evolution', 'minimize', 'nsga2', 'realtime'],
    value='differential_evolution',
    description='Method:',
    layout=Layout(width='100%')
)

n_objectives = IntSlider(
    value=2,
    min=2,
    max=5,
    step=1,
    description='Objectives:',
    layout=Layout(width='100%')
)

max_iterations = IntSlider(
    value=1000,
    min=100,
    max=10000,
    step=100,
    description='Max Iterations:',
    layout=Layout(width='100%')
)

population_size = IntSlider(
    value=50,
    min=10,
    max=200,
    step=10,
    description='Population Size:',
    layout=Layout(width='100%')
)

enable_constraints = Checkbox(
    value=False,
    description='Enable Constraints',
    layout=Layout(width='100%')
)

constraint_method = Dropdown(
    options=['penalty', 'barrier', 'augmented_lagrangian'],
    value='penalty',
    description='Constraint Method:',
    layout=Layout(width='100%')
)

enable_realtime = Checkbox(
    value=False,
    description='Enable Real-Time',
    layout=Layout(width='100%')
)

realtime_update_interval = FloatSlider(
    value=1.0,
    min=0.1,
    max=10.0,
    step=0.1,
    description='Update Interval (s):',
    layout=Layout(width='100%')
)

optimization_config = VBox([
    WidgetHTML("<b>Optimization Configuration</b>"),
    optimization_type,
    optimization_method,
    n_objectives,
    max_iterations,
    population_size,
    enable_constraints,
    constraint_method,
    enable_realtime,
    realtime_update_interval,
], layout=Layout(padding='10px'))

# Validation Configuration
validation_method = RadioButtons(
    options=[('Cross-validation', 'cross_validation'), ('Experimental', 'experimental'), ('Simulation', 'simulation')],
    value='cross_validation',
    description='Method:',
    style={'description_width': 'initial'}
)

validation_folds = IntSlider(
    value=5,
    min=3,
    max=10,
    step=1,
    description='Folds:',
    layout=Layout(width='100%')
)

validation_tolerance = FloatSlider(
    value=0.1,
    min=0.01,
    max=0.5,
    step=0.01,
    description='Tolerance:',
    layout=Layout(width='100%')
)

enable_experimental = Checkbox(
    value=False,
    description='Enable Experimental',
    layout=Layout(width='100%')
)

validation_config = VBox([
    WidgetHTML("<b>Validation Configuration</b>"),
    validation_method,
    validation_folds,
    validation_tolerance,
    enable_experimental,
], layout=Layout(padding='10px'))

# Model Tracking Configuration
enable_model_registry = Checkbox(
    value=True,
    description='Enable Model Registry',
    layout=Layout(width='100%')
)

enable_performance_tracking = Checkbox(
    value=True,
    description='Enable Performance Tracking',
    layout=Layout(width='100%')
)

enable_drift_detection = Checkbox(
    value=True,
    description='Enable Drift Detection',
    layout=Layout(width='100%')
)

drift_threshold = FloatSlider(
    value=0.1,
    min=0.05,
    max=0.3,
    step=0.05,
    description='Drift Threshold:',
    layout=Layout(width='100%')
)

degradation_threshold = FloatSlider(
    value=0.1,
    min=0.05,
    max=0.3,
    step=0.05,
    description='Degradation Threshold:',
    layout=Layout(width='100%')
)

model_tracking_config = VBox([
    WidgetHTML("<b>Model Tracking Configuration</b>"),
    enable_model_registry,
    enable_performance_tracking,
    enable_drift_detection,
    drift_threshold,
    degradation_threshold,
], layout=Layout(padding='10px'))

# Configuration Accordion
config_accordion = Accordion(children=[
    prediction_config,
    optimization_config,
    validation_config,
    model_tracking_config,
], selected_index=None, layout=Layout(width='100%'))

config_accordion.set_title(0, 'Prediction')
config_accordion.set_title(1, 'Optimization')
config_accordion.set_title(2, 'Validation')
config_accordion.set_title(3, 'Model Tracking')

left_panel = VBox([
    WidgetHTML("<h3>Configuration</h3>"),
    config_accordion,
], layout=Layout(width='350px', border='2px solid #f57c00', padding='10px', margin='5px'))

print("✅ Configuration accordion created")

✅ Configuration accordion created


In [5]:
# ============================================
# Center Panel: Main Output
# ============================================

main_output = Output(layout=Layout(height='600px', border='1px solid #ccc', overflow_y='auto'))

# Initialize main output
with main_output:
    display(HTML("<p><i>Results and visualizations will appear here...</i></p>"))

center_panel = VBox([
    WidgetHTML("<h3>Results & Visualizations</h3>"),
    main_output,
], layout=Layout(flex='1 1 auto', border='2px solid #4caf50', padding='10px', margin='5px'))

print("✅ Center panel created")

✅ Center panel created


In [6]:
# ============================================
# Right Panel: Status Displays
# ============================================

# Status Display
status_display = WidgetHTML(
    value='<b>Status:</b> Ready<br><b>Operation:</b> None<br><b>Time:</b> 0:00',
    layout=Layout(height='150px', border='1px solid #ccc', padding='10px')
)

# Model Performance Display
model_performance_display = WidgetHTML(
    value='<b>Model Performance:</b><br>No model trained yet',
    layout=Layout(height='150px', border='1px solid #ccc', padding='10px')
)

# Optimization Results Display
optimization_results_display = WidgetHTML(
    value='<b>Optimization Results:</b><br>No optimization performed yet',
    layout=Layout(height='150px', border='1px solid #ccc', padding='10px')
)

# Model Tracking Display
model_tracking_display = WidgetHTML(
    value='<b>Model Tracking:</b><br>No models registered yet',
    layout=Layout(height='150px', border='1px solid #ccc', padding='10px')
)

right_panel = VBox([
    WidgetHTML("<h3>Status & Metrics</h3>"),
    status_display,
    model_performance_display,
    optimization_results_display,
    model_tracking_display,
], layout=Layout(width='300px', border='2px solid #7b1fa2', padding='10px', margin='5px'))

print("✅ Right panel created")

✅ Right panel created


In [7]:
# ============================================
# Bottom Panel: Progress and Logs
# ============================================

# Progress bar
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Progress:',
    bar_style='info',
    layout=Layout(width='100%')
)

# Status text
status_text = WidgetHTML(value='<b>Status:</b> Ready')

# Processing logs output
processing_logs = Output(layout=Layout(height='200px', border='1px solid #ccc', overflow_y='auto'))

# Initialize logs
with processing_logs:
    display(HTML("<p><i>Processing logs will appear here...</i></p>"))

# Global time tracking
operation_start_time = None

bottom_panel = VBox([
    progress_bar,
    status_text,
    WidgetHTML("<b>Processing Logs:</b>"),
    processing_logs,
], layout=Layout(padding='10px', border='1px solid #ccc', margin='5px'))

print("✅ Bottom panel created")

✅ Bottom panel created


In [8]:
# ============================================
# Logging Functions
# ============================================

def log_message(message: str, level: str = 'info'):
    """Log a message to the processing logs with timestamp and emoji."""
    timestamp = datetime.now().strftime('%H:%M:%S')
    
    if level == 'success':
        emoji = '✅'
        color = 'green'
    elif level == 'warning':
        emoji = '⚠️'
        color = 'orange'
    elif level == 'error':
        emoji = '❌'
        color = 'red'
    else:
        emoji = 'ℹ️'
        color = 'blue'
    
    log_entry = f'<p style="color: {color}; margin: 2px 0;"><b>{emoji} [{timestamp}]</b> {message}</p>'
    
    with processing_logs:
        display(HTML(log_entry), display_id=True)

def update_progress(value: int, message: str = ''):
    """Update progress bar and status."""
    progress_bar.value = value
    if message:
        status_text.value = f'<b>Status:</b> {message} | <b>Progress:</b> {value}%'
    else:
        status_text.value = f'<b>Status:</b> In Progress | <b>Progress:</b> {value}%'

def update_status(operation: str, status: str, elapsed_time: float = None):
    """Update status display."""
    time_str = f"{elapsed_time:.1f}s" if elapsed_time else "0:00"
    status_display.value = f'<b>Status:</b> {status}<br><b>Operation:</b> {operation}<br><b>Time:</b> {time_str}'

def update_model_performance(metrics: Dict[str, float]):
    """Update model performance display."""
    metrics_html = '<br>'.join([f'<b>{k}:</b> {v:.4f}' for k, v in metrics.items()])
    model_performance_display.value = f'<b>Model Performance:</b><br>{metrics_html}'

def update_optimization_results(result: Dict[str, Any]):
    """Update optimization results display."""
    if result:
        params_html = '<br>'.join([f'<b>{k}:</b> {v:.4f}' for k, v in result.get('optimal_parameters', {}).items()])
        value_html = f"<b>Optimal Value:</b> {result.get('optimal_value', 'N/A')}"
        optimization_results_display.value = f'<b>Optimization Results:</b><br>{params_html}<br>{value_html}'
    else:
        optimization_results_display.value = '<b>Optimization Results:</b><br>No optimization performed yet'

def update_model_tracking(info: Dict[str, Any]):
    """Update model tracking display."""
    if info:
        models_html = f"<b>Registered Models:</b> {info.get('registered_count', 0)}<br>"
        models_html += f"<b>Performance Evaluations:</b> {info.get('evaluation_count', 0)}<br>"
        models_html += f"<b>Drift Score:</b> {info.get('drift_score', 0.0):.4f}"
        model_tracking_display.value = f'<b>Model Tracking:</b><br>{models_html}'
    else:
        model_tracking_display.value = '<b>Model Tracking:</b><br>No models registered yet'

print("✅ Logging functions created")

✅ Logging functions created


In [9]:
# ============================================
# Execution Functions
# ============================================

def execute_quality_prediction():
    """Execute predictive quality model training and evaluation."""
    global current_operation, operation_start_time, is_operation_active
    
    try:
        is_operation_active = True
        current_operation = 'Quality Prediction'
        operation_start_time = time.time()
        
        log_message("Starting quality prediction model training...", 'info')
        update_progress(10, "Loading data...")
        update_status(current_operation, "Loading Data")
        
        # Load or generate data
        if data_source_mode.value == 'demo':
            process_data = generate_demo_process_data(n_samples=200)
            log_message(f"Generated demo data: {len(process_data)} samples", 'success')
        else:
            log_message("MongoDB/CSV data loading not implemented in demo", 'warning')
            process_data = generate_demo_process_data(n_samples=200)
        
        update_progress(30, "Training model...")
        log_message("Training quality prediction model...", 'info')
        
        # Configure and train
        if QUALITY_AVAILABLE and quality_predictor:
            feature_names = list(prediction_features.value)
            if not feature_names:
                feature_names = ['laser_power', 'scan_speed', 'layer_thickness', 'hatch_spacing']
            
            result = quality_predictor.analyze_quality_prediction(
                process_data,
                quality_target='quality',
                feature_names=feature_names
            )
            
            update_progress(70, "Evaluating model...")
            
            if result.success:
                log_message(f"Model trained successfully! R²: {result.model_performance.get('r2_score', 0):.4f}", 'success')
                update_model_performance(result.model_performance)
                
                # Register model if enabled
                if enable_model_registry.value and MODEL_TRACKING_AVAILABLE and model_registry and quality_predictor.trained_model:
                    try:
                        model_id = model_registry.register_model(
                            model=quality_predictor.trained_model,
                            model_type=prediction_model_type.value.title().replace('_', ''),
                            version='1.0',
                            metadata={'feature_names': feature_names},
                            performance_metrics=result.model_performance
                        )
                        log_message(f"Model registered: {model_id}", 'success')
                        update_model_tracking({'registered_count': len(model_registry._models)})
                    except Exception as e:
                        log_message(f"Model registration failed: {e}", 'warning')
                
                # Visualize results
                update_progress(90, "Generating visualizations...")
                with main_output:
                    clear_output(wait=True)
                    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
                    
                    # Quality predictions vs actual
                    axes[0, 0].scatter(process_data['quality'].values[:len(result.quality_predictions)], 
                                     result.quality_predictions, alpha=0.6)
                    axes[0, 0].plot([0, 1], [0, 1], 'r--', lw=2)
                    axes[0, 0].set_xlabel('Actual Quality')
                    axes[0, 0].set_ylabel('Predicted Quality')
                    axes[0, 0].set_title('Quality Predictions vs Actual')
                    axes[0, 0].grid(True)
                    
                    # Feature importance (if available)
                    if hasattr(quality_predictor.trained_model, 'feature_importances_'):
                        importances = quality_predictor.trained_model.feature_importances_
                        feature_names_list = feature_names[:len(importances)]
                        axes[0, 1].barh(feature_names_list, importances)
                        axes[0, 1].set_xlabel('Importance')
                        axes[0, 1].set_title('Feature Importance')
                        axes[0, 1].grid(True)
                    
                    # Quality distribution
                    axes[1, 0].hist(result.quality_predictions, bins=20, alpha=0.7, edgecolor='black')
                    axes[1, 0].set_xlabel('Predicted Quality')
                    axes[1, 0].set_ylabel('Frequency')
                    axes[1, 0].set_title('Quality Distribution')
                    axes[1, 0].grid(True)
                    
                    # Model performance metrics
                    metrics = result.model_performance
                    metric_names = list(metrics.keys())[:5]  # Top 5 metrics
                    metric_values = [metrics[k] for k in metric_names]
                    axes[1, 1].bar(metric_names, metric_values)
                    axes[1, 1].set_ylabel('Value')
                    axes[1, 1].set_title('Model Performance Metrics')
                    axes[1, 1].tick_params(axis='x', rotation=45)
                    axes[1, 1].grid(True)
                    
                    plt.tight_layout()
                    plt.show()
                
                update_progress(100, "Complete!")
                elapsed = time.time() - operation_start_time
                update_status(current_operation, "Complete", elapsed)
                log_message(f"Quality prediction completed in {elapsed:.2f}s", 'success')
                
                prediction_results['quality'] = result
            else:
                log_message(f"Model training failed: {result.error_message}", 'error')
                update_progress(100, "Failed")
        else:
            log_message("Quality predictor not available - using demo mode", 'warning')
            update_progress(100, "Demo mode")
        
    except Exception as e:
        log_message(f"Error in quality prediction: {str(e)}", 'error')
        import traceback
        log_message(f"Traceback: {traceback.format_exc()}", 'error')
        update_progress(100, "Error")
    finally:
        is_operation_active = False

def execute_early_defect_detection():
    """Execute early defect detection."""
    global current_operation, operation_start_time, is_operation_active
    
    try:
        is_operation_active = True
        current_operation = 'Early Defect Detection'
        operation_start_time = time.time()
        
        log_message("Starting early defect detection model training...", 'info')
        update_progress(10, "Loading data...")
        
        # Load or generate data
        process_data = generate_demo_process_data(n_samples=200)
        feature_names = list(prediction_features.value) if prediction_features.value else ['laser_power', 'scan_speed', 'layer_thickness', 'hatch_spacing']
        
        update_progress(30, "Training early defect model...")
        
        if PREDICTION_AVAILABLE and early_defect_predictor:
            # Train model
            result = early_defect_predictor.train_early_prediction_model(
                process_data[feature_names],
                process_data['defect_label'].values,
                feature_names=feature_names,
                early_horizon=early_prediction_horizon.value
            )
            
            update_progress(70, "Evaluating predictions...")
            
            if result.success:
                log_message(f"Early defect model trained! Accuracy: {result.early_prediction_accuracy:.4f}", 'success')
                update_model_performance(result.model_performance)
                
                # Test on partial data
                partial_data = process_data[feature_names].iloc[:30]
                defect_prob, confidence = early_defect_predictor.predict_early_defect(
                    partial_data, build_progress=0.3
                )
                
                # Visualize
                update_progress(90, "Generating visualizations...")
                with main_output:
                    clear_output(wait=True)
                    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
                    
                    # Defect probability over time
                    axes[0, 0].plot(defect_prob, 'b-', label='Defect Probability', linewidth=2)
                    axes[0, 0].axhline(y=0.5, color='r', linestyle='--', label='Threshold')
                    axes[0, 0].set_xlabel('Sample Index')
                    axes[0, 0].set_ylabel('Defect Probability')
                    axes[0, 0].set_title('Early Defect Probability')
                    axes[0, 0].legend()
                    axes[0, 0].grid(True)
                    
                    # Prediction confidence
                    axes[0, 1].plot(confidence, 'g-', label='Confidence', linewidth=2)
                    axes[0, 1].set_xlabel('Sample Index')
                    axes[0, 1].set_ylabel('Confidence')
                    axes[0, 1].set_title('Prediction Confidence')
                    axes[0, 1].legend()
                    axes[0, 1].grid(True)
                    
                    # Feature importance
                    if result.feature_importance:
                        top_features = sorted(result.feature_importance.items(), key=lambda x: x[1], reverse=True)[:10]
                        feature_names_list = [f[0] for f in top_features]
                        importances = [f[1] for f in top_features]
                        axes[1, 0].barh(feature_names_list, importances)
                        axes[1, 0].set_xlabel('Importance')
                        axes[1, 0].set_title('Feature Importance for Early Detection')
                        axes[1, 0].grid(True)
                    
                    # Model performance
                    metrics = result.model_performance
                    metric_names = list(metrics.keys())
                    metric_values = [metrics[k] for k in metric_names]
                    axes[1, 1].bar(metric_names, metric_values)
                    axes[1, 1].set_ylabel('Value')
                    axes[1, 1].set_title('Model Performance')
                    axes[1, 1].tick_params(axis='x', rotation=45)
                    axes[1, 1].grid(True)
                    
                    plt.tight_layout()
                    plt.show()
                
                update_progress(100, "Complete!")
                elapsed = time.time() - operation_start_time
                update_status(current_operation, "Complete", elapsed)
                log_message(f"Early defect detection completed in {elapsed:.2f}s", 'success')
                
                prediction_results['early_defect'] = result
            else:
                log_message(f"Early defect model training failed: {result.error_message}", 'error')
        else:
            log_message("Early defect predictor not available", 'warning')
        
    except Exception as e:
        log_message(f"Error in early defect detection: {str(e)}", 'error')
        import traceback
        log_message(f"Traceback: {traceback.format_exc()}", 'error')
    finally:
        is_operation_active = False

def execute_time_series_forecasting():
    """Execute time-series forecasting."""
    global current_operation, operation_start_time, is_operation_active
    
    try:
        is_operation_active = True
        current_operation = 'Time-Series Forecasting'
        operation_start_time = time.time()
        
        log_message("Starting time-series forecasting...", 'info')
        update_progress(20, "Loading data...")
        
        # Generate time-series data
        process_data = generate_demo_process_data(n_samples=100)
        quality_series = process_data['quality'].values
        
        update_progress(40, "Forecasting...")
        
        if PREDICTION_AVAILABLE and time_series_predictor:
            # Forecast using moving average (most reliable)
            result = time_series_predictor.forecast_quality_metric(
                quality_series,
                forecast_horizon=forecast_horizon.value,
                model_type='moving_average'
            )
            
            update_progress(80, "Generating visualizations...")
            
            if result.success:
                log_message(f"Forecast completed! Horizon: {result.forecast_horizon}", 'success')
                
                # Visualize
                with main_output:
                    clear_output(wait=True)
                    fig, axes = plt.subplots(2, 1, figsize=(14, 10))
                    
                    # Historical and forecast
                    historical_len = len(result.historical_data)
                    forecast_len = len(result.forecast)
                    x_historical = np.arange(historical_len)
                    x_forecast = np.arange(historical_len, historical_len + forecast_len)
                    
                    axes[0].plot(x_historical, result.historical_data, 'b-', label='Historical', linewidth=2)
                    axes[0].plot(x_forecast, result.forecast, 'r-', label='Forecast', linewidth=2)
                    axes[0].fill_between(x_forecast, result.forecast_lower_bound, result.forecast_upper_bound, 
                                          alpha=0.3, color='red', label='Confidence Interval')
                    axes[0].axvline(x=historical_len, color='g', linestyle='--', label='Forecast Start')
                    axes[0].set_xlabel('Time Step')
                    axes[0].set_ylabel('Quality')
                    axes[0].set_title('Quality Time-Series Forecast')
                    axes[0].legend()
                    axes[0].grid(True)
                    
                    # Forecast with bounds
                    axes[1].plot(x_forecast, result.forecast, 'r-', label='Forecast', linewidth=2)
                    axes[1].fill_between(x_forecast, result.forecast_lower_bound, result.forecast_upper_bound, 
                                        alpha=0.3, color='red', label='Confidence Interval')
                    axes[1].set_xlabel('Forecast Step')
                    axes[1].set_ylabel('Quality')
                    axes[1].set_title('Forecast with Confidence Intervals')
                    axes[1].legend()
                    axes[1].grid(True)
                    
                    plt.tight_layout()
                    plt.show()
                
                update_progress(100, "Complete!")
                elapsed = time.time() - operation_start_time
                update_status(current_operation, "Complete", elapsed)
                log_message(f"Time-series forecasting completed in {elapsed:.2f}s", 'success')
                
                prediction_results['time_series'] = result
            else:
                log_message(f"Forecasting failed: {result.error_message}", 'error')
        else:
            log_message("Time-series predictor not available", 'warning')
        
    except Exception as e:
        log_message(f"Error in time-series forecasting: {str(e)}", 'error')
        import traceback
        log_message(f"Traceback: {traceback.format_exc()}", 'error')
    finally:
        is_operation_active = False

print("✅ Execution functions created (partial)")

✅ Execution functions created (partial)


In [10]:
# Continue execution functions

def execute_optimization_single():
    """Execute single-objective optimization."""
    global current_operation, operation_start_time, is_operation_active
    
    try:
        is_operation_active = True
        current_operation = 'Single-Objective Optimization'
        operation_start_time = time.time()
        
        log_message("Starting single-objective optimization...", 'info')
        update_progress(10, "Preparing data and model...")
        
        # First train quality predictor if not already trained
        process_data = generate_demo_process_data(n_samples=200)
        feature_names = ['laser_power', 'scan_speed', 'layer_thickness', 'hatch_spacing']
        
        if QUALITY_AVAILABLE and quality_predictor:
            if quality_predictor.trained_model is None:
                log_message("Training quality predictor for optimization...", 'info')
                quality_predictor.analyze_quality_prediction(
                    process_data, quality_target='quality', feature_names=feature_names
                )
            
            update_progress(30, "Defining objective function...")
            
            # Define objective function
            def objective_function(params):
                param_df = pd.DataFrame([params])
                quality_pred = quality_predictor.predict_quality(param_df)
                return -quality_pred[0]  # Negative for minimization
            
            # Parameter bounds
            parameter_bounds = {
                'laser_power': (200.0, 300.0),
                'scan_speed': (800.0, 1200.0),
                'layer_thickness': (0.02, 0.04),
                'hatch_spacing': (0.08, 0.12)
            }
            
            update_progress(50, "Running optimization...")
            log_message(f"Optimization method: {optimization_method.value}, Max iterations: {max_iterations.value}", 'info')
            
            if OPTIMIZATION_AVAILABLE and optimizer:
                # Configure optimizer
                optimizer.config.max_iterations = max_iterations.value
                optimizer.config.population_size = population_size.value
                optimizer.config.optimization_method = optimization_method.value
                
                # Handle constraints if enabled
                if enable_constraints.value:
                    def energy_constraint(params):
                        return (params['laser_power'] / params['scan_speed']) - 0.3
                    
                    result = optimizer.optimize_with_constraints(
                        objective_function,
                        parameter_bounds,
                        [energy_constraint],
                        constraint_method=constraint_method.value
                    )
                else:
                    result = optimizer.optimize_single_objective(
                        objective_function,
                        parameter_bounds
                    )
                
                update_progress(90, "Generating visualizations...")
                
                if result.success:
                    log_message(f"Optimization completed! Optimal value: {result.optimal_values:.4f}", 'success')
                    
                    opt_result_dict = {
                        'optimal_parameters': result.optimal_parameters,
                        'optimal_value': -result.optimal_values  # Convert back to positive
                    }
                    update_optimization_results(opt_result_dict)
                    optimization_results['single'] = result
                    
                    # Visualize
                    with main_output:
                        clear_output(wait=True)
                        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
                        
                        # Optimal parameters
                        param_names = list(result.optimal_parameters.keys())
                        param_values = list(result.optimal_parameters.values())
                        axes[0, 0].bar(param_names, param_values)
                        axes[0, 0].set_ylabel('Value')
                        axes[0, 0].set_title('Optimal Parameters')
                        axes[0, 0].tick_params(axis='x', rotation=45)
                        axes[0, 0].grid(True)
                        
                        # Parameter comparison (optimal vs bounds)
                        bounds_mid = [(parameter_bounds[k][0] + parameter_bounds[k][1]) / 2 for k in param_names]
                        axes[0, 1].barh([f"{n}_optimal" for n in param_names], param_values, alpha=0.7, label='Optimal')
                        axes[0, 1].barh([f"{n}_mid" for n in param_names], bounds_mid, alpha=0.5, label='Bounds Mid')
                        axes[0, 1].set_xlabel('Value')
                        axes[0, 1].set_title('Optimal vs Bounds Midpoint')
                        axes[0, 1].legend()
                        axes[0, 1].grid(True)
                        
                        # Optimization history (if available)
                        if result.optimization_history:
                            axes[1, 0].plot(result.optimization_history, 'b-', linewidth=2)
                            axes[1, 0].set_xlabel('Iteration')
                            axes[1, 0].set_ylabel('Objective Value')
                            axes[1, 0].set_title('Optimization Convergence')
                            axes[1, 0].grid(True)
                        
                        # Quality prediction at optimal parameters
                        optimal_df = pd.DataFrame([result.optimal_parameters])
                        optimal_quality = quality_predictor.predict_quality(optimal_df)[0]
                        axes[1, 1].bar(['Optimal Quality'], [optimal_quality], color='green', alpha=0.7)
                        axes[1, 1].axhline(y=0.8, color='r', linestyle='--', label='Target (0.8)')
                        axes[1, 1].set_ylabel('Quality')
                        axes[1, 1].set_title('Predicted Quality at Optimal Parameters')
                        axes[1, 1].legend()
                        axes[1, 1].grid(True)
                        
                        plt.tight_layout()
                        plt.show()
                    
                    update_progress(100, "Complete!")
                    elapsed = time.time() - operation_start_time
                    update_status(current_operation, "Complete", elapsed)
                    log_message(f"Single-objective optimization completed in {elapsed:.2f}s", 'success')
                else:
                    log_message(f"Optimization failed: {result.error_message}", 'error')
            else:
                log_message("Optimizer not available", 'warning')
        else:
            log_message("Quality predictor not available", 'warning')
        
    except Exception as e:
        log_message(f"Error in optimization: {str(e)}", 'error')
        import traceback
        log_message(f"Traceback: {traceback.format_exc()}", 'error')
    finally:
        is_operation_active = False

def execute_optimization_multi():
    """Execute multi-objective optimization."""
    global current_operation, operation_start_time, is_operation_active
    
    try:
        is_operation_active = True
        current_operation = 'Multi-Objective Optimization'
        operation_start_time = time.time()
        
        log_message("Starting multi-objective optimization...", 'info')
        update_progress(20, "Defining objectives...")
        
        # Define multiple objectives
        def objective_functions(params):
            # Objective 1: Maximize quality
            quality = 0.7 + 0.001 * params['laser_power'] - 10 * params['layer_thickness']
            quality = np.clip(quality, 0.0, 1.0)
            
            # Objective 2: Minimize energy consumption
            energy = params['laser_power'] / params['scan_speed']
            
            return [-quality, energy]  # Negative quality for minimization
        
        parameter_bounds = {
            'laser_power': (200.0, 300.0),
            'scan_speed': (800.0, 1200.0),
            'layer_thickness': (0.02, 0.04)
        }
        
        update_progress(40, "Running multi-objective optimization...")
        
        if OPTIMIZATION_AVAILABLE and optimizer:
            optimizer.config.max_iterations = max_iterations.value
            optimizer.config.population_size = population_size.value
            optimizer.config.n_objectives = n_objectives.value
            optimizer.config.pareto_front_size = 50
            
            result = optimizer.optimize_multi_objective(
                objective_functions,
                parameter_bounds,
                n_objectives=n_objectives.value
            )
            
            update_progress(80, "Generating visualizations...")
            
            if result.success and result.pareto_front is not None and len(result.pareto_front) > 0:
                log_message(f"Multi-objective optimization completed! Pareto solutions: {len(result.pareto_front)}", 'success')
                
                # Extract Pareto front
                pareto_df = result.pareto_front
                
                # Visualize
                with main_output:
                    clear_output(wait=True)
                    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
                    
                    # Pareto front (if objectives available)
                    if 'objectives' in pareto_df.columns:
                        objectives_list = pareto_df['objectives'].tolist()
                        if len(objectives_list) > 0 and isinstance(objectives_list[0], (list, np.ndarray)):
                            obj1 = [-o[0] for o in objectives_list]  # Quality (negate back)
                            obj2 = [o[1] for o in objectives_list]  # Energy
                            
                            axes[0, 0].scatter(obj2, obj1, alpha=0.6, s=50)
                            axes[0, 0].set_xlabel('Energy Consumption')
                            axes[0, 0].set_ylabel('Quality')
                            axes[0, 0].set_title('Pareto Front: Quality vs Energy')
                            axes[0, 0].grid(True)
                    
                    # Parameter distributions in Pareto front
                    if 'parameters' in pareto_df.columns:
                        params_list = pareto_df['parameters'].tolist()
                        if len(params_list) > 0:
                            laser_powers = [p[0] if isinstance(p, (list, np.ndarray)) else 0 for p in params_list]
                            scan_speeds = [p[1] if isinstance(p, (list, np.ndarray)) and len(p) > 1 else 0 for p in params_list]
                            
                            axes[0, 1].scatter(laser_powers, scan_speeds, alpha=0.6, s=50)
                            axes[0, 1].set_xlabel('Laser Power')
                            axes[0, 1].set_ylabel('Scan Speed')
                            axes[0, 1].set_title('Parameter Space in Pareto Front')
                            axes[0, 1].grid(True)
                    
                    # Pareto front size
                    axes[1, 0].bar(['Pareto Solutions'], [len(pareto_df)], color='green', alpha=0.7)
                    axes[1, 0].set_ylabel('Number of Solutions')
                    axes[1, 0].set_title('Pareto Front Size')
                    axes[1, 0].grid(True)
                    
                    # Solution distribution
                    axes[1, 1].hist(range(len(pareto_df)), bins=min(20, len(pareto_df)), alpha=0.7, edgecolor='black')
                    axes[1, 1].set_xlabel('Solution Index')
                    axes[1, 1].set_ylabel('Frequency')
                    axes[1, 1].set_title('Solution Distribution')
                    axes[1, 1].grid(True)
                    
                    plt.tight_layout()
                    plt.show()
                
                optimization_results['multi'] = result
                update_progress(100, "Complete!")
                elapsed = time.time() - operation_start_time
                update_status(current_operation, "Complete", elapsed)
                log_message(f"Multi-objective optimization completed in {elapsed:.2f}s", 'success')
            else:
                log_message("Multi-objective optimization failed or no Pareto solutions", 'error')
        else:
            log_message("Optimizer not available", 'warning')
        
    except Exception as e:
        log_message(f"Error in multi-objective optimization: {str(e)}", 'error')
        import traceback
        log_message(f"Traceback: {traceback.format_exc()}", 'error')
    finally:
        is_operation_active = False

print("✅ Optimization execution functions created")

✅ Optimization execution functions created


In [11]:
# Continue with validation, model tracking, and complete workflow functions

def execute_validation():
    """Execute optimization validation."""
    global current_operation, operation_start_time, is_operation_active
    
    try:
        is_operation_active = True
        current_operation = 'Optimization Validation'
        operation_start_time = time.time()
        
        log_message("Starting optimization validation...", 'info')
        update_progress(10, "Preparing validation...")
        
        process_data = generate_demo_process_data(n_samples=200)
        feature_names = ['laser_power', 'scan_speed', 'layer_thickness', 'hatch_spacing']
        
        # Train quality predictor
        if QUALITY_AVAILABLE and quality_predictor:
            if quality_predictor.trained_model is None:
                quality_predictor.analyze_quality_prediction(
                    process_data, quality_target='quality', feature_names=feature_names
                )
            
            update_progress(30, "Performing validation...")
            
            if PREDICTION_AVAILABLE and prediction_validator:
                if validation_method.value == 'cross_validation':
                    log_message("Performing cross-validation...", 'info')
                    cv_result = prediction_validator.cross_validate_model(
                        quality_predictor,
                        process_data,
                        quality_target='quality',
                        n_folds=validation_folds.value,
                        validation_method='kfold'
                    )
                    
                    update_progress(80, "Generating visualizations...")
                    
                    with main_output:
                        clear_output(wait=True)
                        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
                        
                        # CV metrics
                        if 'mean_r2' in cv_result:
                            metrics = ['mean_r2', 'mean_rmse', 'mean_mae']
                            means = [cv_result.get(m, 0) for m in metrics]
                            stds = [cv_result.get(m.replace('mean', 'std'), 0) for m in metrics]
                            
                            x = np.arange(len(metrics))
                            axes[0].bar(x, means, yerr=stds, alpha=0.7, capsize=5)
                            axes[0].set_xticks(x)
                            axes[0].set_xticklabels(metrics, rotation=45)
                            axes[0].set_ylabel('Value')
                            axes[0].set_title('Cross-Validation Metrics')
                            axes[0].grid(True)
                        
                        # Validation summary
                        summary_text = f"Validation Method: {cv_result.get('validation_method', 'N/A')}<br>"
                        summary_text += f"Folds: {cv_result.get('n_folds', 'N/A')}<br>"
                        summary_text += f"Analysis Time: {cv_result.get('analysis_time', 0):.2f}s"
                        axes[1].text(0.1, 0.5, summary_text, fontsize=12, verticalalignment='center',
                                    transform=axes[1].transAxes)
                        axes[1].axis('off')
                        axes[1].set_title('Validation Summary')
                        
                        plt.tight_layout()
                        plt.show()
                    
                    log_message(f"Cross-validation completed! Mean R²: {cv_result.get('mean_r2', 0):.4f}", 'success')
                
                elif validation_method.value == 'experimental':
                    log_message("Performing experimental validation...", 'info')
                    
                    # Simulate experimental data
                    predicted_data = process_data.iloc[:20].copy()
                    experimental_data = pd.DataFrame({
                        'quality': process_data['quality'].iloc[:20].values + np.random.randn(20) * 0.05
                    })
                    
                    validation_result = prediction_validator.validate_with_experimental_data(
                        quality_predictor,
                        predicted_data,
                        experimental_data,
                        quality_target='quality'
                    )
                    
                    if validation_result.success:
                        log_message(f"Experimental validation completed! Error: {validation_result.validation_error:.4f}", 'success')
                        
                        with main_output:
                            clear_output(wait=True)
                            fig, axes = plt.subplots(1, 2, figsize=(14, 5))
                            
                            # Predicted vs Experimental
                            axes[0].scatter(validation_result.experimental_objective, 
                                          validation_result.predicted_objective, alpha=0.6)
                            axes[0].plot([0, 1], [0, 1], 'r--', lw=2)
                            axes[0].set_xlabel('Experimental Quality')
                            axes[0].set_ylabel('Predicted Quality')
                            axes[0].set_title('Predicted vs Experimental')
                            axes[0].grid(True)
                            
                            # Validation metrics
                            metrics = validation_result.validation_metrics
                            metric_names = list(metrics.keys())
                            metric_values = [metrics[k] for k in metric_names]
                            axes[1].bar(metric_names, metric_values)
                            axes[1].set_ylabel('Value')
                            axes[1].set_title('Validation Metrics')
                            axes[1].tick_params(axis='x', rotation=45)
                            axes[1].grid(True)
                            
                            plt.tight_layout()
                            plt.show()
                
                update_progress(100, "Complete!")
                elapsed = time.time() - operation_start_time
                update_status(current_operation, "Complete", elapsed)
                log_message(f"Validation completed in {elapsed:.2f}s", 'success')
            else:
                log_message("Prediction validator not available", 'warning')
        else:
            log_message("Quality predictor not available", 'warning')
        
    except Exception as e:
        log_message(f"Error in validation: {str(e)}", 'error')
        import traceback
        log_message(f"Traceback: {traceback.format_exc()}", 'error')
    finally:
        is_operation_active = False

def execute_model_tracking():
    """Execute model tracking operations."""
    global current_operation, operation_start_time, is_operation_active
    
    try:
        is_operation_active = True
        current_operation = 'Model Tracking'
        operation_start_time = time.time()
        
        log_message("Starting model tracking operations...", 'info')
        update_progress(20, "Loading models...")
        
        if MODEL_TRACKING_AVAILABLE and model_registry:
            # List all models
            all_models = model_registry.list_models()
            log_message(f"Found {len(all_models)} registered models", 'info')
            
            if len(all_models) > 0:
                update_progress(40, "Evaluating model performance...")
                
                # Get first model for tracking
                model_id = all_models[0]['model_id']
                model, model_version = model_registry.load_model(model_id)
                
                # Create tracker
                tracker = ModelPerformanceTracker(
                    model_id=model_id,
                    model_registry=model_registry,
                    history_size=10
                )
                
                # Generate test data
                test_data = generate_demo_process_data(n_samples=30)
                
                # Evaluate performance
                metrics = tracker.evaluate_model_performance(
                    model=model,
                    test_data=test_data,
                    quality_target='quality'
                )
                
                update_progress(60, "Calculating drift...")
                
                # Calculate drift
                training_data = generate_demo_process_data(n_samples=100)
                drift_score = tracker.calculate_drift_score(
                    test_data[['laser_power', 'scan_speed', 'layer_thickness', 'hatch_spacing']],
                    training_data[['laser_power', 'scan_speed', 'layer_thickness', 'hatch_spacing']]
                )
                
                update_progress(80, "Generating visualizations...")
                
                # Get performance trend
                trend = tracker.get_performance_trend('r2_score' if 'r2_score' in metrics.performance_metrics else 'mae')
                
                # Update displays
                update_model_performance(metrics.performance_metrics)
                update_model_tracking({
                    'registered_count': len(all_models),
                    'evaluation_count': len(tracker.performance_history),
                    'drift_score': drift_score
                })
                
                # Visualize
                with main_output:
                    clear_output(wait=True)
                    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
                    
                    # Model performance metrics
                    perf_metrics = metrics.performance_metrics
                    metric_names = list(perf_metrics.keys())[:5]
                    metric_values = [perf_metrics[k] for k in metric_names]
                    axes[0, 0].bar(metric_names, metric_values)
                    axes[0, 0].set_ylabel('Value')
                    axes[0, 0].set_title('Model Performance Metrics')
                    axes[0, 0].tick_params(axis='x', rotation=45)
                    axes[0, 0].grid(True)
                    
                    # Performance history
                    history = tracker.get_performance_history()
                    if len(history) > 0:
                        r2_scores = [h.get('performance_metrics', {}).get('r2_score', 0) for h in history if 'r2_score' in h.get('performance_metrics', {})]
                        if r2_scores:
                            axes[0, 1].plot(r2_scores, 'b-o', linewidth=2, markersize=8)
                            axes[0, 1].set_xlabel('Evaluation')
                            axes[0, 1].set_ylabel('R² Score')
                            axes[0, 1].set_title('Performance History')
                            axes[0, 1].grid(True)
                    
                    # Drift score
                    axes[1, 0].bar(['Drift Score'], [drift_score], color='orange', alpha=0.7)
                    axes[1, 0].axhline(y=drift_threshold.value, color='r', linestyle='--', label=f'Threshold ({drift_threshold.value})')
                    axes[1, 0].set_ylabel('Drift Score')
                    axes[1, 0].set_title('Data Drift Detection')
                    axes[1, 0].legend()
                    axes[1, 0].grid(True)
                    
                    # Performance trend
                    if trend.get('trend_available'):
                        trend_dir = trend.get('trend_direction', 'unknown')
                        axes[1, 1].text(0.5, 0.5, f"Trend: {trend_dir}\nSlope: {trend.get('slope', 0):.4f}",
                                       fontsize=14, ha='center', va='center', transform=axes[1, 1].transAxes)
                        axes[1, 1].set_title('Performance Trend')
                    else:
                        axes[1, 1].text(0.5, 0.5, "Insufficient data for trend",
                                       fontsize=12, ha='center', va='center', transform=axes[1, 1].transAxes)
                        axes[1, 1].set_title('Performance Trend')
                    axes[1, 1].axis('off')
                    
                    plt.tight_layout()
                    plt.show()
                
                update_progress(100, "Complete!")
                elapsed = time.time() - operation_start_time
                update_status(current_operation, "Complete", elapsed)
                log_message(f"Model tracking completed in {elapsed:.2f}s", 'success')
            else:
                log_message("No models registered. Train a model first.", 'warning')
        else:
            log_message("Model tracking not available", 'warning')
        
    except Exception as e:
        log_message(f"Error in model tracking: {str(e)}", 'error')
        import traceback
        log_message(f"Traceback: {traceback.format_exc()}", 'error')
    finally:
        is_operation_active = False

def execute_complete_workflow():
    """Execute complete end-to-end workflow."""
    global current_operation, operation_start_time, is_operation_active
    
    try:
        is_operation_active = True
        current_operation = 'Complete Workflow'
        operation_start_time = time.time()
        
        log_message("Starting complete prediction and optimization workflow...", 'info')
        update_progress(5, "Step 1: Training quality predictor...")
        
        # Step 1: Train quality predictor
        process_data = generate_demo_process_data(n_samples=200)
        feature_names = ['laser_power', 'scan_speed', 'layer_thickness', 'hatch_spacing']
        
        if QUALITY_AVAILABLE and quality_predictor:
            quality_result = quality_predictor.analyze_quality_prediction(
                process_data, quality_target='quality', feature_names=feature_names
            )
            
            if quality_result.success:
                log_message("Step 1 complete: Quality predictor trained", 'success')
                update_progress(20, "Step 2: Registering model...")
                
                # Step 2: Register model
                if enable_model_registry.value and MODEL_TRACKING_AVAILABLE and model_registry:
                    model_id = model_registry.register_model(
                        model=quality_predictor.trained_model,
                        model_type='RandomForestRegressor',
                        version='1.0',
                        metadata={'feature_names': feature_names},
                        performance_metrics=quality_result.model_performance
                    )
                    log_message(f"Step 2 complete: Model registered ({model_id})", 'success')
                
                update_progress(40, "Step 3: Optimizing parameters...")
                
                # Step 3: Optimize
                if OPTIMIZATION_AVAILABLE and optimizer:
                    def objective_function(params):
                        param_df = pd.DataFrame([params])
                        quality_pred = quality_predictor.predict_quality(param_df)
                        return -quality_pred[0]
                    
                    parameter_bounds = {
                        'laser_power': (200.0, 300.0),
                        'scan_speed': (800.0, 1200.0),
                        'layer_thickness': (0.02, 0.04),
                        'hatch_spacing': (0.08, 0.12)
                    }
                    
                    opt_result = optimizer.optimize_single_objective(
                        objective_function,
                        parameter_bounds
                    )
                    
                    if opt_result.success:
                        log_message("Step 3 complete: Optimization completed", 'success')
                        optimization_results['workflow'] = opt_result
                        
                        update_progress(70, "Step 4: Validating results...")
                        
                        # Step 4: Validate
                        if PREDICTION_AVAILABLE and prediction_validator:
                            predicted_data = pd.DataFrame([opt_result.optimal_parameters])
                            experimental_data = pd.DataFrame({'quality': [0.85]})
                            
                            validation_result = prediction_validator.validate_with_experimental_data(
                                quality_predictor,
                                predicted_data,
                                experimental_data,
                                quality_target='quality'
                            )
                            
                            if validation_result.success:
                                log_message("Step 4 complete: Validation completed", 'success')
                        
                        update_progress(90, "Generating visualizations...")
                        
                        # Visualize complete workflow
                        with main_output:
                            clear_output(wait=True)
                            fig, axes = plt.subplots(2, 2, figsize=(14, 10))
                            
                            # Quality predictions
                            axes[0, 0].scatter(process_data['quality'].values[:len(quality_result.quality_predictions)], 
                                             quality_result.quality_predictions, alpha=0.6)
                            axes[0, 0].plot([0, 1], [0, 1], 'r--', lw=2)
                            axes[0, 0].set_xlabel('Actual Quality')
                            axes[0, 0].set_ylabel('Predicted Quality')
                            axes[0, 0].set_title('Quality Predictions')
                            axes[0, 0].grid(True)
                            
                            # Optimal parameters
                            param_names = list(opt_result.optimal_parameters.keys())
                            param_values = list(opt_result.optimal_parameters.values())
                            axes[0, 1].bar(param_names, param_values)
                            axes[0, 1].set_ylabel('Value')
                            axes[0, 1].set_title('Optimal Parameters')
                            axes[0, 1].tick_params(axis='x', rotation=45)
                            axes[0, 1].grid(True)
                            
                            # Model performance
                            metrics = quality_result.model_performance
                            metric_names = list(metrics.keys())[:5]
                            metric_values = [metrics[k] for k in metric_names]
                            axes[1, 0].bar(metric_names, metric_values)
                            axes[1, 0].set_ylabel('Value')
                            axes[1, 0].set_title('Model Performance')
                            axes[1, 0].tick_params(axis='x', rotation=45)
                            axes[1, 0].grid(True)
                            
                            # Workflow summary
                            summary = f"Models Registered: {len(model_registry._models) if model_registry else 0}<br>"
                            summary += f"Optimal Quality: {-opt_result.optimal_values:.4f}<br>"
                            summary += f"Validation Error: {validation_result.validation_error:.4f if 'validation_result' in locals() else 'N/A'}"
                            axes[1, 1].text(0.5, 0.5, summary, fontsize=12, ha='center', va='center',
                                          transform=axes[1, 1].transAxes)
                            axes[1, 1].set_title('Workflow Summary')
                            axes[1, 1].axis('off')
                            
                            plt.tight_layout()
                            plt.show()
                        
                        update_progress(100, "Complete!")
                        elapsed = time.time() - operation_start_time
                        update_status(current_operation, "Complete", elapsed)
                        log_message(f"Complete workflow finished in {elapsed:.2f}s", 'success')
                    else:
                        log_message("Optimization failed in workflow", 'error')
            else:
                log_message("Quality prediction failed in workflow", 'error')
        else:
            log_message("Required components not available", 'warning')
        
    except Exception as e:
        log_message(f"Error in complete workflow: {str(e)}", 'error')
        import traceback
        log_message(f"Traceback: {traceback.format_exc()}", 'error')
    finally:
        is_operation_active = False

print("✅ All execution functions created")

✅ All execution functions created


In [12]:
# ============================================
# Main Execution Handler
# ============================================

def on_execute_button_clicked(b):
    """Handle execute button click."""
    global is_operation_active
    
    if is_operation_active:
        log_message("Operation already in progress. Please wait or stop current operation.", 'warning')
        return
    
    op_type = operation_type.value
    
    # Clear previous results
    with main_output:
        clear_output(wait=True)
    
    # Route to appropriate execution function
    if op_type == 'quality_prediction':
        execute_quality_prediction()
    elif op_type == 'early_defect':
        execute_early_defect_detection()
    elif op_type == 'time_series':
        execute_time_series_forecasting()
    elif op_type == 'optimization_single':
        execute_optimization_single()
    elif op_type == 'optimization_multi':
        execute_optimization_multi()
    elif op_type == 'validation':
        execute_validation()
    elif op_type == 'realtime_optimization':
        log_message("Real-time optimization not fully implemented in demo", 'warning')
        execute_optimization_single()  # Fallback to single-objective
    elif op_type == 'model_tracking':
        execute_model_tracking()
    elif op_type == 'complete':
        execute_complete_workflow()
    else:
        log_message(f"Unknown operation type: {op_type}", 'error')

def on_stop_button_clicked(b):
    """Handle stop button click."""
    global is_operation_active
    is_operation_active = False
    log_message("Operation stopped by user", 'warning')
    update_progress(0, "Stopped")
    stop_button.disabled = True
    execute_button.disabled = False

def on_export_button_clicked(b):
    """Handle export button click."""
    try:
        export_data = {
            'prediction_results': {k: str(v) for k, v in prediction_results.items()},
            'optimization_results': {k: str(v) for k, v in optimization_results.items()},
            'timestamp': datetime.now().isoformat()
        }
        
        export_file = f"optimization_prediction_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        with open(export_file, 'w') as f:
            json.dump(export_data, f, indent=2, default=str)
        
        log_message(f"Results exported to {export_file}", 'success')
    except Exception as e:
        log_message(f"Export failed: {e}", 'error')

# Attach event handlers
execute_button.on_click(on_execute_button_clicked)
stop_button.on_click(on_stop_button_clicked)
export_button.on_click(on_export_button_clicked)

print("✅ Event handlers attached")

✅ Event handlers attached


In [13]:
# ============================================
# Display Complete Interface
# ============================================

# Main layout: Top, then Left-Center-Right, then Bottom
main_layout = VBox([
    top_panel,
    HBox([
        left_panel,
        center_panel,
        right_panel,
    ], layout=Layout(width='100%', height='650px')),
    bottom_panel,
], layout=Layout(width='100%', padding='10px'))

display(main_layout)

log_message("Interactive Process Optimization and Prediction Interface ready!", 'success')
log_message("Select an operation type and click 'Execute Operation' to begin.", 'info')